El proyecto Magia es un proyecto que busca medir como responde la gente frente a una situacion novedosa y dificil de explicar en terminos fisicos. La idea es hacer un "truco" (el del buzo en una botella de agua) donde no es obvio porque el globo sube o baja, disfrazado de algo "sorprendente" y pedirle a la gente que complete una encuesta sobre que cree que esta pasando. Buscamos ver cuanto inventa la gente y con que sesgo en funcion de su formacion profesional. 

Los datos de la encuesta son levantados automaticamente mediante el sotware sdaps que genera los formularios y despues mediante escaneo los procesa en bloque sin necesidad de pasar los datos a manos a una compu. Del programa se puede exportar un csv con los datos en bruto. Donde a cada entrada le corresponde una encuesta completada. 

In [23]:
import pandas
from IPython.display import display

En el siguiente bloque cargamos los datos y los procesamos para que sean mas legibles. Para eso hay que interpretar el csv donde viene guardados. El [csv](./DataValidada_PilotoKonex.csv) se genero automaticamente por el spdaps a partir de escanear el [archivo](./questionnaire.pdf), donde como se puede ver, hay secciones, preguntas y opciones multiples. Cada seccion esta indicada en el encabezado por un primer numero, cada preguntas por un segundo numero y cada opcion por un tercero unidos con un _ luego cada fila representa una entrada donde un 1 representa seleccionado en el caso de las opciones multiples o el valor seleccionado en caso de que se tratara de una escala. 

In [157]:
data = pandas.read_csv('./DataValidada_PilotoKonex.csv')
inter = interprete('Interprete.txt')
data.drop(data.columns[[0, 1,]], axis=1, inplace=True) # Son columnas que no tienen informacion valida

for columna in data.columns:
    indices = columna.split('_')
    indices = [int(valor) for valor in indices]
    if len (indices) == 2:
        old = columna
        new = inter[indices[0]][indices[1]]['Pregunta']
        data.rename(columns={columna:inter[indices[0]][indices[1]]['Pregunta']}, inplace=True)
    if len (indices) == 3:
        pregunta = inter[indices[0]][indices[1]]['Pregunta']
        if indices[2] == 0:
            data[pregunta] = [[]] * len(data)
        opcion = inter[indices[0]][indices[1]][indices[2]+1]['Opcion']
        data[pregunta] = [x if y == 0 else x + [opcion] for x,y in zip(data[pregunta],data[columna])]  
        data.drop([columna], axis=1, inplace=True)

In [156]:
data.drop(['1_1_0'], axis=1, inplace=True)

In [158]:
data.head()

,Nivel_seguridad,Formacion,Justificacion,Truco
0,4,[Biomedicas],[Telequinesis],[Si]
1,10,[Ciencias Sociales],[No lo se],[No se]
2,0,[],[],[]
3,7,[Artes],[Magnetismo],[No]
4,8,[Artes],[Presion],[No se]


In [91]:
data.columns

Index(['1_1_0', '1_1_1', '1_1_2', '1_1_3', '1_1_4', '1_1_5', '1_1_6', '1_1_7',
       '1_1_8', '1_1_9', '1_1_10', '2_1_0', '2_1_1', '2_1_2', '2_1_3', '2_1_4',
       '2_1_5', '2_1_6', '2_1_7', '3_1', '3_2_0', '3_2_1', '3_2_2'],
      dtype='object')

In [69]:
## Esta funcion importa un archivo donde en cada linea hay una seccion/pregunta/opcion 
## usando un nivel de identacion diferente para cada categoria y a partir de eso crea un dict
## donde usando los indices de seccion - pregunta - opcion se puede saber los textos correspondientes
## Esto es porque en los archivo creados y escaneados usando sdps guarda las respuestas como una variable 
## con una etiqueta antes que indica el numero de seccion - pregunta - opcion pero no guarda los textos correspondientes
def interprete (filename):
    
    import re

    secciones = 0
    preguntas = 0
    opciones = 0
    nivelActual = 0
    interprete = {}

    with open(filename) as f:
        lineas = f.readlines()
    for linea in lineas:
        tabs = len(re.findall("\t", linea))
        if tabs < nivelActual:
            if tabs == 1:
                opciones = 0
            if tabs == 0:
                preguntas = 0
                opciones = 0
        if tabs == 0:
            seccion = {}
            secciones = secciones + 1
            seccion['Titulo'] = linea[:-1]
            interprete[secciones] = seccion
        if tabs == 1:
            nivelActual = 1
            pregunta = {}
            preguntas += 1
            pregunta['Pregunta'] = linea[1:-1]
            interprete[secciones][preguntas] = pregunta
        if tabs == 2:
            nivelActual = 2
            opcion = {}
            opciones += 1
            opcion['Opcion'] = linea[2:-1]
            interprete[secciones][preguntas][opciones] = opcion
            
    return interprete

In [20]:
display (interprete)

{1: {'Indice': 1,
  'Preguntas': {1: {'Indice': 1,
    'Pregunta': 'Formacion',
    'Opciones': {1: {'Indice': 1, 'Opcion': 'Artes'},
     2: {'Indice': 2, 'Opcion': 'Biomedicas'},
     3: {'Indice': 3, 'Opcion': 'Economicas'},
     4: {'Indice': 4, 'Opcion': 'Ciencias Exactas'},
     5: {'Indice': 5, 'Opcion': 'Fisica'},
     6: {'Indice': 6, 'Opcion': 'Ciencias Sociales'},
     7: {'Indice': 7, 'Opcion': 'Derecho'},
     8: {'Indice': 8, 'Opcion': 'Otra Carrera'},
     9: {'Indice': 9, 'Opcion': 'Psicologia'},
     10: {'Indice': 10, 'Opcion': 'Ninguna'},
     11: {'Indice': 11, 'Opcion': 'Otro'}}}},
  'Titulo': 'Formacion'},
 2: {'Indice': 2,
  'Preguntas': {1: {'Indice': 1,
    'Pregunta': 'Tiene que ver con...',
    'Opciones': {1: {'Indice': 1, 'Opcion': 'Calor'},
     2: {'Indice': 2, 'Opcion': 'Telequinesis'},
     3: {'Indice': 3, 'Opcion': 'Magnetismo'},
     4: {'Indice': 4, 'Opcion': 'Energia Cosmica'},
     5: {'Indice': 5, 'Opcion': 'Presion'},
     6: {'Indice': 6, 'Opci